## Segmenting and Clustering Neighborhoods in the City of Toronto, Canada

Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be UPDATED:

  ca-certificates      anaconda::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2020.4.5.2-hecda079_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi               anaconda::certifi-2020.4.5.2-py36_0 --> conda-forge::certifi-2020.4.5.2-py36h9f0ad1d_0
  openssl               anaconda::openssl-1.1.1g-h7b6447c_0 --> conda-forge::openssl-1.1.1g-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


In [2]:
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c anaconda lxml --yes
from bs4 import BeautifulSoup
print("done")

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> anaconda::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5.2-py36h~ --> anaconda::certifi-2020.4.5.2-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> anaconda::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

# All requested packages already installed.

done


### Cleaning Data to create a Dataframe 'df' for Toronto, Canada

In [3]:
#send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
page = BeautifulSoup(data,'html.parser')

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []
 

In [4]:
# append the data into the respective lists
for row in page.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [5]:
# create a new DataFrame from the three lists
df = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [6]:
df.shape

(180, 3)

In [7]:
# drop cells with value Not assigned
df_dropna = df[df.Borough != "Not assigned\n"].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [8]:
df_dropna.shape

(103, 3)

## Answer 1

In [21]:
# group neighborhoods in the same borough
df_group = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

In [23]:
# removing extra spaces to make both dataframes similar
df_group['Neighborhood'] = df_group['Neighborhood'].str.replace(' /', ',')
df_group['PostalCode'] = df_group['PostalCode'].str.replace('\n', '')
df_group['Borough'] = df_group['Borough'].str.replace('\n', '')
df_group.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [24]:
df_group.shape

(103, 3)

#### Using Csv file to get Coordinates

In [10]:
# getting longitude and latitudes
csv_path = 'https://cocl.us/Geospatial_data'
df_ll = pd.read_csv(csv_path)
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_ll.shape

(103, 3)

In [12]:
df_ll.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_ll.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df_group.shape

(103, 3)

In [17]:
df_ll.shape

(103, 3)

In [18]:
df_group.iloc[0,0] == df_ll.iloc[0,0]

True

### Answer 2

In [25]:
#merging dataframes
toronto_df = df_group.merge(df_ll, on="PostalCode", how='left')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
toronto_df.shape

(103, 5)

In [27]:
toronto_df[toronto_df['Borough'].astype(str).str.contains('Toronto')]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


Getting Toronto location using 'Geocode'

In [28]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Toronto Map using Folium

In [29]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Defining Foursquare Credentials


In [30]:
CLIENT_ID = '1CV351J0MTT0CJ1USL4QPYJ1CR4D2D4LIICWJI3V5V02ZH3Y' # Foursquare ID
CLIENT_SECRET = 'LWHBS4NKKNDE4D3C2MEB2TSIAHOQX2IP4QWJUBXW03EGEZUZ' # Foursquare Secret
VERSION = '20200618' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1CV351J0MTT0CJ1USL4QPYJ1CR4D2D4LIICWJI3V5V02ZH3Y
CLIENT_SECRET:LWHBS4NKKNDE4D3C2MEB2TSIAHOQX2IP4QWJUBXW03EGEZUZ


Exploring Neighborhood of Toronto

In [31]:
LIMIT = 10
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
#to get new dataframe containing venues in neighborhood
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [34]:
print(toronto_venues.shape)
toronto_venues.head()

(688, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [35]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",10,10,10,10,10,10
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",10,10,10,10,10,10


In [36]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 180 uniques categories.


##### Analyze each neighborhood

In [51]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[124]] + list(toronto_onehot.columns[:])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot = toronto_onehot.loc[:,~toronto_onehot.columns.duplicated()]

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [52]:
toronto_onehot.shape

(688, 180)

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [53]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Noodle House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.250,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.25,0.000000,0.000000,0.25,0.000000,0.000,0.00,0.000000,0.000,0.000,0.000000,0.00,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0000,0.000000,0.0,0.250,0.00,0.0000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.00,0.000,0.0,0.0,0.0,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
1,"Alderwood, Long Branch",0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.142857,0.0,0.0,0.000000,0.000000,0.000000,0.0000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.142857,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.00,0.000000,0

In [54]:
toronto_grouped.shape

(94, 180)

#### Getting top 10 venues in each Neighborhoof and putting in Dataframe

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Coffee Shop,Athletics & Sports,Sandwich Place,Discount Store,Falafel Restaurant,Event Space,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bridal Shop,Bank,Deli / Bodega,Middle Eastern Restaurant,Diner,Ice Cream Shop,Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Comfort Food Restaurant,Coffee Shop,Pub,Juice Bar,Café,Indian Restaurant,Sushi Restaurant,Thai Restaurant


## Clustering Neighborhoods

### Answer 3

In [112]:
# set number of clusters
kclusters = 6

toronto_group_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_group_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 2, 2, 2, 2, 2, 2, 3, 1], dtype=int32)

In [113]:
neighborhoods_venues_sorted.drop(['Cluster Labels'],axis= 1,inplace=True)
neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
1,"Alderwood, Long Branch",Pizza Place,Gym,Pub,Coffee Shop,Athletics & Sports,Sandwich Place,Discount Store,Falafel Restaurant,Event Space,Electronics Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bridal Shop,Bank,Deli / Bodega,Middle Eastern Restaurant,Diner,Ice Cream Shop,Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
4,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Comfort Food Restaurant,Coffee Shop,Pub,Juice Bar,Café,Indian Restaurant,Sushi Restaurant,Thai Restaurant


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [114]:
#adding Cluster labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3,Fast Food Restaurant,Yoga Studio,Convenience Store,Field,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4,Bar,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Breakfast Spot,Medical Center,Mexican Restaurant,Rental Car Location,Electronics Store,Moving Target,Bank,Intersection,Drugstore,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Gas Station,Caribbean Restaurant,Discount Store,Distribution Center


Visualizing Clusters

In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clsuters

#### Cluster 1

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0,Playground,Park,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run
23,North York,0,Convenience Store,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
25,North York,0,Construction & Landscaping,Food & Drink Shop,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
40,East York,0,Park,Convenience Store,Metro Station,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
44,Central Toronto,0,Park,Swim School,Bus Line,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
50,Downtown Toronto,0,Park,Playground,Trail,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
74,York,0,Park,Women's Store,Pool,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
79,North York,0,Construction & Landscaping,Park,Basketball Court,Bakery,Yoga Studio,Drugstore,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
98,York,0,Convenience Store,Park,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore


#### Cluster 2

In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,1,Coffee Shop,Korean Restaurant,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
29,North York,1,Caribbean Restaurant,Metro Station,Falafel Restaurant,Miscellaneous Shop,Coffee Shop,Bar,Massage Studio,Drugstore,Fast Food Restaurant,Farmers Market
57,Downtown Toronto,1,Coffee Shop,Modern European Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Spa,Yoga Studio,Farmers Market,Falafel Restaurant
68,Downtown Toronto,1,Airport Lounge,Harbor / Marina,Plane,Coffee Shop,Boutique,Bar,Airport Terminal,Airport Food Court,Airport,Field
80,York,1,Sandwich Place,Bar,Turkish Restaurant,Discount Store,Yoga Studio,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
86,Mississauga,1,Coffee Shop,Hotel,Mediterranean Restaurant,Burrito Place,Middle Eastern Restaurant,American Restaurant,Fried Chicken Joint,Sandwich Place,Drugstore,Farmers Market


#### Cluster 3

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,2,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bank,Bakery,Thai Restaurant,Gas Station,Caribbean Restaurant,Discount Store,Distribution Center
6,Scarborough,2,Convenience Store,Department Store,Bus Station,Chinese Restaurant,Train Station,Coffee Shop,Discount Store,Deli / Bodega,Cuban Restaurant,Fast Food Restaurant
7,Scarborough,2,Bus Line,Bakery,Intersection,Ice Cream Shop,Bus Station,Soccer Field,Park,Department Store,Eastern European Restaurant,Fast Food Restaurant
9,Scarborough,2,Café,College Stadium,Skating Rink,General Entertainment,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
10,Scarborough,2,Indian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Light Rail Station,Brewery,Pet Store,Curling Ice,Dance Studio,Fast Food Restaurant,Farmers Market
11,Scarborough,2,Smoke Shop,Middle Eastern Restaurant,Bakery,Breakfast Spot,Yoga Studio,Drugstore,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
18,North York,2,Restaurant,Bakery,Electronics Store,Burger Joint,Gift Shop,Shopping Mall,Movie Theater,Salon / Barbershop,Toy / Game Store,Tea Room
19,North York,2,Japanese Restaurant,Bank,Chinese Restaurant,Café,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
22,North York,2,Café,Indonesian Restaurant,Grocery Store,Steakhouse,Coffee Shop,Arts & Crafts Store,Ramen Restaurant,Ice Cream Shop,Movie Theater,Distribution Center
26,North York,2,Japanese Restaurant,Gym,Coffee Shop,Sporting Goods Shop,Bike Shop,Café,Caribbean Restaurant,Italian Restaurant,Clothing Store,Restaurant


#### Cluster 4

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Yoga Studio,Convenience Store,Field,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore
2,Scarborough,3,Breakfast Spot,Medical Center,Mexican Restaurant,Rental Car Location,Electronics Store,Moving Target,Bank,Intersection,Drugstore,Fast Food Restaurant
5,Scarborough,3,Playground,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run
8,Scarborough,3,Motel,American Restaurant,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
12,Scarborough,3,Breakfast Spot,Lounge,Latin American Restaurant,Skating Rink,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
13,Scarborough,3,Pizza Place,Gas Station,Fried Chicken Joint,Bank,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Rental Car Location,Fast Food Restaurant,Noodle House
15,Scarborough,3,Chinese Restaurant,Breakfast Spot,Bank,Pizza Place,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Grocery Store,Gay Bar
17,North York,3,Mediterranean Restaurant,Pool,Golf Course,Dog Run,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
24,North York,3,Pizza Place,Pharmacy,Grocery Store,Coffee Shop,Bank,Discount Store,Distribution Center,Farmers Market,Falafel Restaurant,Event Space
34,North York,3,Intersection,Pizza Place,Portuguese Restaurant,Hockey Arena,Coffee Shop,French Restaurant,Discount Store,Falafel Restaurant,Event Space,Electronics Store


#### CLuster 5

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4,Bar,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
102,Etobicoke,4,Rental Car Location,Bar,Drugstore,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store


#### Cluster 6

In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,5,Home Service,Garden,Yoga Studio,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
91,Etobicoke,5,Home Service,Baseball Field,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store
97,North York,5,Baseball Field,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
